In [43]:
#vzorka_cela.crosstab("Accident_Severity", "Day_of_Week").show()

+-----------------------------+-----+-----+-----+-----+-----+-----+-----+
|Accident_Severity_Day_of_Week|    1|    2|    3|    4|    5|    6|    7|
+-----------------------------+-----+-----+-----+-----+-----+-----+-----+
|                            2|23942|29487|30915|30958|31556|34732|28784|
|                            1|  649|  542|  500|  495|  539|  741|  737|
+-----------------------------+-----+-----+-----+-----+-----+-----+-----+



In [ ]:
#vzorka_cela.saveAsTextFile('small_data.csv')

In [ ]:
#vzorka_cela.write.format('com.databricks.spark.csv').mode('append').option("header", "true").save('C:/Users/FIlip/Documents/TSVD/Zadanie')

In [2]:
import sys
from pyspark import SparkContext

sc = SparkContext(appName="example23")

print(u'Python version ' + sys.version)
print(u'Spark version ' + sc.version)


Python version 2.7.13 (v2.7.13:a06454b1afa1, Dec 17 2016, 20:53:40) [MSC v.1500 64 bit (AMD64)]
Spark version 2.2.0


In [3]:
from pyspark.sql import SparkSession
from pyspark.mllib.random import RandomRDDs
from pyspark.sql.types import *
from pyspark.sql import SQLContext
import sys
from pyspark import SparkContext
spark = SparkSession.builder.appName("Zadanie").getOrCreate()


sqlContext = SQLContext(sc)




In [90]:
# nacitanie
data=spark.read.format('csv').options(header='true', inferSchema='true').load('C:/Users/FIlip/Documents/TSVD/Zadanie/dataset/Accidents.csv')
data1=spark.read.format('csv').options(header='true', inferSchema='true').load('C:/Users/FIlip/Documents/TSVD/Zadanie/dataset/Vehicles.csv')
data2=spark.read.format('csv').options(header='true', inferSchema='true').load('C:/Users/FIlip/Documents/TSVD/Zadanie/dataset/Casualties.csv')

In [91]:
data1=data1.withColumnRenamed("Accident_Index","ID")
data2=data2.withColumnRenamed("Accident_Index","IDE")

In [92]:
#spojenie dat a vymazanie potom tych duplikatych ID stlpcov
merge = data.join(data1, data.Accident_Index == data1.ID)
merge.drop("ID")
full_merge = merge.join(data2, merge.Accident_Index == data2.IDE)
full_merge=full_merge.drop("IDE")
full_merge=full_merge.drop("ID")
full_merge=full_merge.drop("Vehicle_Reference")

len(full_merge.columns)
full_merge
#print(full_merge.show())

DataFrame[Accident_Index: string, Location_Easting_OSGR: int, Location_Northing_OSGR: int, Longitude: double, Latitude: double, Police_Force: int, Accident_Severity: int, Number_of_Vehicles: int, Number_of_Casualties: int, Date: string, Day_of_Week: int, Time: string, Local_Authority_(District): int, Local_Authority_(Highway): string, 1st_Road_Class: int, 1st_Road_Number: int, Road_Type: int, Speed_limit: int, Junction_Detail: int, Junction_Control: int, 2nd_Road_Class: int, 2nd_Road_Number: int, Pedestrian_Crossing-Human_Control: int, Pedestrian_Crossing-Physical_Facilities: int, Light_Conditions: int, Weather_Conditions: int, Road_Surface_Conditions: int, Special_Conditions_at_Site: int, Carriageway_Hazards: int, Urban_or_Rural_Area: int, Did_Police_Officer_Attend_Scene_of_Accident: int, LSOA_of_Accident_Location: string, Vehicle_Type: int, Towing_and_Articulation: int, Vehicle_Manoeuvre: int, Vehicle_Location-Restricted_Lane: int, Junction_Location: int, Skidding_and_Overturning: in

In [34]:
# čiže ci cieľovy atribut je Accident_Severity a ma 3 hodnoty : 1, 2, 3 - jedna je smrtelna 2 je že važna ale nezomrel a 3 je že ľahka... tak som 3 replacol na 2 aby bolo len smrtelna
# a niesmrtelna
from pyspark.sql.functions import when
newsdf = full_merge.withColumn("Accident_Severity", when(full_merge["Accident_Severity"] == 3, 2).otherwise(full_merge["Accident_Severity"]))

In [36]:
#cize sampling  som si vytiahol koľko je počet mrtvych a koľko počet čo prežili nehodu a podľa toho som v tokom pomere zmenšil tie data na 5%
newsdf.registerTempTable("TempTable")
mrtvy = sqlContext.sql('SELECT * FROM TempTable WHERE Accident_Severity = 1')
mrtvy_pocet = mrtvy.count()
zivy = sqlContext.sql('SELECT * FROM TempTable WHERE Accident_Severity = 2')
zivy_pocet = zivy.count()

vzorka_mrtvy = mrtvy.sampleBy("Accident_Severity", fractions = {1: 0.05}, seed = 0)
vzorka_zivy = zivy.sampleBy("Accident_Severity", fractions = {2: 0.05}, seed = 0)

#tu ich nazad spojim a s tym datasetom treba pracovať ďalej
vzorka_cela = vzorka_mrtvy.union(vzorka_zivy)


In [37]:
#rozdelenie na trenovaciu a testovaciu 
training_data, test_data = vzorka_cela.randomSplit([0.6, 0.4], seed=123)

In [38]:
print(vzorka_cela.take(1))
print("------")

[Row(Accident_Index=u'200522DB43935', Location_Easting_OSGR=385650, Location_Northing_OSGR=273200, Longitude=-2.212142, Latitude=52.356639, Police_Force=22, Accident_Severity=1, Number_of_Vehicles=2, Number_of_Casualties=3, Date=u'03/04/2005', Day_of_Week=1, Time=u'15:41', Local_Authority_(District)=278, Local_Authority_(Highway)=u'E10000034', 1st_Road_Class=3, 1st_Road_Number=450, Road_Type=6, Speed_limit=60, Junction_Detail=0, Junction_Control=-1, 2nd_Road_Class=-1, 2nd_Road_Number=0, Pedestrian_Crossing-Human_Control=0, Pedestrian_Crossing-Physical_Facilities=0, Light_Conditions=1, Weather_Conditions=1, Road_Surface_Conditions=1, Special_Conditions_at_Site=0, Carriageway_Hazards=0, Urban_or_Rural_Area=2, Did_Police_Officer_Attend_Scene_of_Accident=1, LSOA_of_Accident_Location=u'E01032431', Vehicle_Type=9, Towing_and_Articulation=0, Vehicle_Manoeuvre=17, Vehicle_Location-Restricted_Lane=0, Junction_Location=0, Skidding_and_Overturning=0, Hit_Object_in_Carriageway=0, Vehicle_Leaving_C

In [39]:
print(training_data.take(1))
print("-----------")

[Row(Accident_Index=u'200522DB43935', Location_Easting_OSGR=385650, Location_Northing_OSGR=273200, Longitude=-2.212142, Latitude=52.356639, Police_Force=22, Accident_Severity=1, Number_of_Vehicles=2, Number_of_Casualties=3, Date=u'03/04/2005', Day_of_Week=1, Time=u'15:41', Local_Authority_(District)=278, Local_Authority_(Highway)=u'E10000034', 1st_Road_Class=3, 1st_Road_Number=450, Road_Type=6, Speed_limit=60, Junction_Detail=0, Junction_Control=-1, 2nd_Road_Class=-1, 2nd_Road_Number=0, Pedestrian_Crossing-Human_Control=0, Pedestrian_Crossing-Physical_Facilities=0, Light_Conditions=1, Weather_Conditions=1, Road_Surface_Conditions=1, Special_Conditions_at_Site=0, Carriageway_Hazards=0, Urban_or_Rural_Area=2, Did_Police_Officer_Attend_Scene_of_Accident=1, LSOA_of_Accident_Location=u'E01032431', Vehicle_Type=9, Towing_and_Articulation=0, Vehicle_Manoeuvre=17, Vehicle_Location-Restricted_Lane=0, Junction_Location=0, Skidding_and_Overturning=0, Hit_Object_in_Carriageway=0, Vehicle_Leaving_C

In [40]:
print(test_data.take(1))

[Row(Accident_Index=u'200532D088205', Location_Easting_OSGR=498070, Location_Northing_OSGR=389760, Longitude=-0.526598, Latitude=53.395558, Police_Force=32, Accident_Severity=1, Number_of_Vehicles=2, Number_of_Casualties=9, Date=u'18/12/2005', Day_of_Week=1, Time=u'18:00', Local_Authority_(District)=356, Local_Authority_(Highway)=u'E10000019', 1st_Road_Class=3, 1st_Road_Number=631, Road_Type=6, Speed_limit=60, Junction_Detail=0, Junction_Control=-1, 2nd_Road_Class=-1, 2nd_Road_Number=0, Pedestrian_Crossing-Human_Control=0, Pedestrian_Crossing-Physical_Facilities=0, Light_Conditions=6, Weather_Conditions=1, Road_Surface_Conditions=1, Special_Conditions_at_Site=0, Carriageway_Hazards=0, Urban_or_Rural_Area=2, Did_Police_Officer_Attend_Scene_of_Accident=1, LSOA_of_Accident_Location=u'E01026411', Vehicle_Type=9, Towing_and_Articulation=0, Vehicle_Manoeuvre=16, Vehicle_Location-Restricted_Lane=0, Junction_Location=0, Skidding_and_Overturning=0, Hit_Object_in_Carriageway=0, Vehicle_Leaving_C

In [48]:
# #najprv si dropnem tie atributy ktore tam nechcem mat

vzorka_cela   = vzorka_cela.drop(
					 "Local_Authority_(Highway)",
 					 "LSOA_of_Accident_Location",
					 "Time",
					 "Date",
					 "Accident_Index")

In [49]:
#do premennej names si ulozim mena atributov
names = vzorka_cela.schema.names
print(names)

#print(names.count)
#vytvorim si prazdny list correlations a do neho vo for cykle hodim korelacie kazdeho atributu s #atributom accident_severity, ale dajak to nefujguje zatial :D

correlations = []
for name in names:
    correlations.extend([vzorka_cela.stat.corr('Accident_Severity',name)])

['Location_Easting_OSGR', 'Location_Northing_OSGR', 'Longitude', 'Latitude', 'Police_Force', 'Accident_Severity', 'Number_of_Vehicles', 'Number_of_Casualties', 'Day_of_Week', 'Local_Authority_(District)', '1st_Road_Class', '1st_Road_Number', 'Road_Type', 'Speed_limit', 'Junction_Detail', 'Junction_Control', '2nd_Road_Class', '2nd_Road_Number', 'Pedestrian_Crossing-Human_Control', 'Pedestrian_Crossing-Physical_Facilities', 'Light_Conditions', 'Weather_Conditions', 'Road_Surface_Conditions', 'Special_Conditions_at_Site', 'Carriageway_Hazards', 'Urban_or_Rural_Area', 'Did_Police_Officer_Attend_Scene_of_Accident', 'Vehicle_Type', 'Towing_and_Articulation', 'Vehicle_Manoeuvre', 'Vehicle_Location-Restricted_Lane', 'Junction_Location', 'Skidding_and_Overturning', 'Hit_Object_in_Carriageway', 'Vehicle_Leaving_Carriageway', 'Hit_Object_off_Carriageway', '1st_Point_of_Impact', 'Was_Vehicle_Left_Hand_Drive?', 'Journey_Purpose_of_Driver', 'Sex_of_Driver', 'Age_of_Driver', 'Age_Band_of_Driver', 'En

In [52]:
print(correlations)

[0.014786428056535498, -0.01621638844788299, 0.014881649693193376, -0.015886097377899047, -0.03761958333347922, 1.0, -0.06369110692696994, -0.12066636855785096, -0.0031986647821147804, -0.03773716887909429, 0.05359870288344655, 0.006687154872887749, -0.0061092550690142735, -0.11069335025575402, 0.05187111068738223, 0.07414108021980693, 0.074425157616837, 0.020083848899277203, 0.006897313588318909, 0.025252285899736885, -0.04905971443910568, -0.010991163680965898, -0.004272140184665985, 0.005466851521745527, -0.010512147510305124, -0.09896599141093315, 0.05657186361984556, -0.02126233930954928, -0.02388275653412301, -0.05971450692822326, -0.008074087360074804, 0.05314717297823556, -0.051924075960947655, -0.021768269545694333, -0.06959285988824364, -0.05299760866808702, 0.02486474760408348, 0.014137892928534177, -0.0034677890495716777, 0.037079759866389436, -0.031875756336561374, -0.03147575804289195, -0.04003647188244287, -0.012458325547380389, -0.006748442902453038, -0.0012217630387663

In [93]:
names = vzorka_cela.schema.names
#t = zip(names,correlations)
#print(t)
#print("-------------------")
tt = spark.createDataFrame(t)
#tt.show()

tt.registerTempTable("TempTable")
atributy_table = sqlContext.sql('SELECT * FROM TempTable WHERE _2 > 0.05 OR _2 <-0.05')
atributy = atributy_table.select("_2")
atributy_table.show()

+--------------------+--------------------+
|                  _1|                  _2|
+--------------------+--------------------+
|   Accident_Severity|                 1.0|
|  Number_of_Vehicles|-0.06369110692696994|
|Number_of_Casualties|-0.12066636855785096|
|      1st_Road_Class| 0.05359870288344655|
|         Speed_limit|-0.11069335025575402|
|     Junction_Detail| 0.05187111068738223|
|    Junction_Control| 0.07414108021980693|
|      2nd_Road_Class|   0.074425157616837|
| Urban_or_Rural_Area|-0.09896599141093315|
|Did_Police_Office...| 0.05657186361984556|
|   Vehicle_Manoeuvre|-0.05971450692822326|
|   Junction_Location| 0.05314717297823556|
|Skidding_and_Over...|-0.05192407596094...|
|Vehicle_Leaving_C...|-0.06959285988824364|
|Hit_Object_off_Ca...|-0.05299760866808702|
|  Casualty_Reference|-0.10583313411571113|
|   Casualty_Severity|  0.4239262407172667|
+--------------------+--------------------+



In [94]:
cast = atributy_table.take(100)
print(cast)

[Row(_1=u'Accident_Severity', _2=1.0), Row(_1=u'Number_of_Vehicles', _2=-0.06369110692696994), Row(_1=u'Number_of_Casualties', _2=-0.12066636855785096), Row(_1=u'1st_Road_Class', _2=0.05359870288344655), Row(_1=u'Speed_limit', _2=-0.11069335025575402), Row(_1=u'Junction_Detail', _2=0.05187111068738223), Row(_1=u'Junction_Control', _2=0.07414108021980693), Row(_1=u'2nd_Road_Class', _2=0.074425157616837), Row(_1=u'Urban_or_Rural_Area', _2=-0.09896599141093315), Row(_1=u'Did_Police_Officer_Attend_Scene_of_Accident', _2=0.05657186361984556), Row(_1=u'Vehicle_Manoeuvre', _2=-0.05971450692822326), Row(_1=u'Junction_Location', _2=0.05314717297823556), Row(_1=u'Skidding_and_Overturning', _2=-0.051924075960947655), Row(_1=u'Vehicle_Leaving_Carriageway', _2=-0.06959285988824364), Row(_1=u'Hit_Object_off_Carriageway', _2=-0.05299760866808702), Row(_1=u'Casualty_Reference', _2=-0.10583313411571113), Row(_1=u'Casualty_Severity', _2=0.4239262407172667)]


In [ ]:
import numpy as np 
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt 
from scipy.stats import norm 

f, ax = plt.subplots(figsize =(9, 8)) 
cast2.heatmap(corrmat, ax = ax, cmap ="YlGnBu", linewidths = 0.1) 